## Convolution Layer Example

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    ## 조건1: conv layer 2개 & fc layer 2개
    ## 조건2: kernel size=5, stride=1로 통일
    ## output을 참고하여 차원을 계산해주세요!
    ## 답안 ##
    self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
    self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)

    self.fc1 = nn.Linear(10*12*12, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    print("연산 전", x.size())
    x = F.relu(self.conv1(x))
    print("conv1 연산 후", x.size())
    x = F.relu(self.conv2(x))
    print("conv2 연산 후",x.size())
    x = x.view(-1, 10 * 12 * 12)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))  # Input Size: (10, 1, 20, 20)

연산 전 torch.Size([10, 1, 20, 20])
conv1 연산 후 torch.Size([10, 3, 16, 16])
conv2 연산 후 torch.Size([10, 10, 12, 12])
차원 감소 후 torch.Size([10, 1440])
fc1 연산 후 torch.Size([10, 50])
fc2 연산 후 torch.Size([10, 10])


## Max Pooling Layer Example

In [14]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()

    ## 조건1: 최대풀링 layer 2개 & fc layer 2개
    ## 조건2: kernel size=2로 통일, stride는 따로 지정할 필요 없음
    ## output을 참고하여 차원을 계산해주세요!
    ## 답안 ##
    self.max_pool1 = nn.MaxPool2d(kernel_size=2)
    self.max_pool2 = nn.MaxPool2d(kernel_size=2)

    # 출력 = [batch_size, channels(filters), height(feature map의), width(feature map의)]
    # 텐서 차원을 flatten : 모든 채널 값을 연결하여 하나의 긴 벡터로 만듦 : num_of_channel * featuremap_height * featuremap_width가 입력 특성 의미
    self.fc1 = nn.Linear(1*5*5, 50)
    self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
    print("연산 전", x.size())
    x = F.relu(self.max_pool1(x))
    print("max_pool1 연산 후", x.size())
    x = F.relu(self.max_pool2(x))
    print("max_pool2 연산 후",x.size())
    x = x.view(-1, 1 * 5 * 5)
    print("차원 감소 후", x.size())
    x = F.relu(self.fc1(x))
    print("fc1 연산 후", x.size())
    x = self.fc2(x)
    print("fc2 연산 후", x.size())
    return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))

연산 전 torch.Size([10, 1, 20, 20])
max_pool1 연산 후 torch.Size([10, 1, 10, 10])
max_pool2 연산 후 torch.Size([10, 1, 5, 5])
차원 감소 후 torch.Size([10, 25])
fc1 연산 후 torch.Size([10, 50])
fc2 연산 후 torch.Size([10, 10])


## MNIST 데이터셋 train

In [5]:
!pip install torchvision

In [6]:
import torch.optim as optim
from torchvision import datasets, transforms

In [7]:
train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 학습 데이터
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])) # 테스트 데이터
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9.91M/9.91M [00:00<00:00, 127MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28.9k/28.9k [00:00<00:00, 57.2MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>

100%|██████████| 1.65M/1.65M [00:00<00:00, 56.2MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4.54k/4.54k [00:00<00:00, 11.8MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [16]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        ## 조건1: conv layer 2개 & fc layer 2개
            ## 첫 번째 conv layer: 입력 채널 1, 출력 채널 20
            ## 두 번째 fc layer: 출력 채널 10
        ## 조건2: kernel_size=5, stride=1
        ## 답안 ##
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)

        self.fc1 = nn.Linear(50*4*4, 100)
        self.fc2 = nn.Linear(100,10)

    def forward(self, x):

        ## 조건1: relu -> max_pool2d -> relu -> max_pool2d 순서로 이루어짐
        ## 조건2: kernel_size=2, stride=2
        ## 답안 ##

        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)

        x = x.view(-1, 4 * 4 * 50) # [batch_size, 50, 4, 4]
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [17]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

In [18]:
cnn.train()  # 학습을 위함
for epoch in range(10):
  for index, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()  # 기울기 초기화
    output = cnn(data)
    loss = criterion(output, target)
    loss.backward()  # 역전파
    optimizer.step()

    if index % 100 == 0:
      print("loss of {} epoch, {} index : {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index : 2.3310346603393555
loss of 0 epoch, 100 index : 1.8741552829742432
loss of 0 epoch, 200 index : 0.79622483253479
loss of 0 epoch, 300 index : 0.3523216247558594
loss of 0 epoch, 400 index : 0.3535895049571991
loss of 0 epoch, 500 index : 0.27999988198280334
loss of 0 epoch, 600 index : 0.3966509997844696
loss of 0 epoch, 700 index : 0.19621142745018005
loss of 0 epoch, 800 index : 0.19175271689891815
loss of 0 epoch, 900 index : 0.1777612715959549
loss of 0 epoch, 1000 index : 0.20749084651470184
loss of 0 epoch, 1100 index : 0.1623641848564148
loss of 1 epoch, 0 index : 0.2868010103702545
loss of 1 epoch, 100 index : 0.07397087663412094
loss of 1 epoch, 200 index : 0.1917475163936615
loss of 1 epoch, 300 index : 0.08407120406627655
loss of 1 epoch, 400 index : 0.07646617293357849
loss of 1 epoch, 500 index : 0.0812513455748558
loss of 1 epoch, 600 index : 0.07928760349750519
loss of 1 epoch, 700 index : 0.24564042687416077
loss of 1 epoch, 800 index : 0.0766

In [19]:
cnn.eval()  # test case 학습 방지를 위함
test_loss = 0
correct = 0
with torch.no_grad():
  for data, target in test_loader:
    output = cnn(data)
    test_loss += criterion(output, target).item() # sum up batch loss
    pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Test set: Average loss: 6.5174, Accuracy: 9889/10000 (99%)

